# Azure Machine Learning Interpretability SDK による要因探索

品質を予測する機械学習モデルによって製造工程のデータから製造品の品質を予測することが可能になります。それだけでなく、モデルの構造を理解することで、不良に影響を与える説明変数・因子を特定し、不良の原因を見つける手助けができます。本Notebookでは、**Factory.csv** を利用し、製造工程のデータから品質を予測する機械学習を構築し、**Azure Machine Learning Interpretability SDK** の、品質に対する因子の影響度を分析します。

## 1. Python SDK のインポート
Azure Machine Learning service の Python SDKをインポートします。

In [ ]:
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun
import os

In [ ]:
print("Azure ML SDK Version: ", azureml.core.VERSION)

### Azure ML workspace との接続
Azure Machine Learning service との接続を行います。Azure に対する認証が必要です。

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep = '\t')

# 2. 学習データの準備

In [ ]:
import pandas as pd
#os.makedirs("./outputs", exist_ok=True)
df = pd.read_csv('./data/Factory.csv')

In [ ]:
df.tail(10)

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=["Quality","ID"],axis=1)
y = df["Quality"].values

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1,random_state=100,stratify=y)

# 3. 事前設定 (Automated Machine Learning)

In [ ]:
Automl_config = AutoMLConfig(task = 'classification',
                             primary_metric = 'AUC_weighted',
                             iteration_timeout_minutes = 10,
                             iterations = 5,
                             experiment_exit_score = 0.999,
                             blacklist_models = ['KNN'],
                             X = X_train,
                             y = y_train,
                             n_cross_validations = 3)

# 4. 実行と結果確認

In [ ]:
experiment=Experiment(ws, "automlQC_explain")
local_run = experiment.submit(Automl_config, show_output=True)

In [ ]:
from azureml.widgets import RunDetails
RunDetails(local_run).show()

In [ ]:
best_run, fitted_model = local_run.get_output()
best_run

In [ ]:
fitted_model

# 5. Azure Machine Learning Interpretability SDK

[Azure Machine Learning Interpretability SDK](https://docs.microsoft.com/en-US/azure/machine-learning/service/machine-learning-interpretability-explainability?view=azuremgmtcompute-fluent-1.0.0) は、Microsoftと主要な3rd Partyのライブラリ(LIME,SHAP etc)で構成されたモデル解釈のフレームワークで、統合APIをご提供しています。

<img src="https://docs.microsoft.com/en-US/azure/machine-learning/service/media/machine-learning-interpretability-explainability/interpretability-architecture.png#lightbox" width=800 align=left>

In [ ]:
!jupyter nbextension install --py --sys-prefix azureml.contrib.explain.model.visualize
!jupyter nbextension enable --py --sys-prefix azureml.contrib.explain.model.visualize

In [ ]:
from azureml.explain.model.tabular_explainer import TabularExplainer
classes = ["false","true"]
tabular_explainer = TabularExplainer(fitted_model, X_train, features=X_train.columns, classes=classes)

In [ ]:
global_explanation = tabular_explainer.explain_global(X_train[:100])

In [ ]:
from azureml.contrib.explain.model.visualize import ExplanationDashboard
ExplanationDashboard(global_explanation, fitted_model, X_test)